## Imports

In [ ]:
print("Starting") # en demograficos tengo el GENDER
import os
#import dask.dataframe as dd
import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import boto3
import pickle

from sklearn.metrics import mean_squared_error,average_precision_score
import joblib
import seaborn as sns
from bayes_opt import BayesianOptimization
#import lightgbm as lgb
import warnings
from sklearn.preprocessing import label_binarize
from multiprocessing import  cpu_count,Pool
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
from dateutil.relativedelta import relativedelta 
from lifetimes.utils import summary_data_from_transaction_data
import re
from unicodedata import normalize

import gc
from sklearn.utils import class_weight

import pyarrow

import time
import shutil

import sys
sys.path.append(os.path.dirname(os.path.expanduser(".")))
sys.path.append(os.path.dirname(os.path.expanduser("..")))
sys.path.append(os.path.dirname(os.path.expanduser("../defines")))
sys.path.append(os.path.dirname(os.path.expanduser("./utils.py")))
from defines import *

TARGET_COL = "frequency_eval"
TRAIN = True
RANDOM_STATE = 42

from etl_functions import *
from preprocessing import *
from drift_utils import *
from train_utils import train_combinations,simple_fit,foo_evaluation_regression, foo_model_regression,foo_predict_regression,anti_dummies,caster,picture

MODEL_FILES = ['ASP.pkl','clusters.pkl','model_0.pkl','model_1.pkl','model_2.pkl','model_3.pkl',
           'features_model_0.csv','features_model_1.csv','features_model_2.csv','features_model_3.csv']  

EXCLUDE_VARIABLES_DRIFTED = {0:[],1:[],2:[],3:[]}

In [ ]:
if os.environ['SECRET_DEBUG'] == 'True':
    print("DEBUG: True")
    exit(0)

In [ ]:
#Cliente BigQuery

from google_cloud import BigQuery
from google_cloud import Storage
import base64
import os
bq = BigQuery(AUTH_BIGQUERY)
bqs = Storage(AUTH_BIGQUERY)

In [ ]:
# Generacion de fechas:
# 2 dias random en el periodo from_ : to_ por cada dia de la semana, para procurar un
# balance de transacciones salvando las diferencias que hay en compras entre dias habiles
# y fines de semana

to_ = FECHA_END - relativedelta(months=FORECAST_MONTHS) - relativedelta(days=1)
from_ = to_ - relativedelta(months = 2)

fechas_queries = [] 

for j in range(7):
    dias = np.random.choice(list(range(8)),size=2,replace = False)
    aux = from_ + relativedelta(weeks = int(dias[0]))
    aux += relativedelta(days=1+j) 
    aux += relativedelta(months=FORECAST_MONTHS)  
    fechas_queries.append(aux)
    aux = from_ + relativedelta(weeks = int(dias[1]))
    aux += relativedelta(days=1+j) 
    aux += relativedelta(months=FORECAST_MONTHS)  
    fechas_queries.append(aux)

pd.DataFrame([x.strftime("%Y_%m_%d") for x in fechas_queries],columns = ["Fecha"]).to_csv("fechas.csv",index=False,sep="|")
bqs.upload_file('fechas.csv',gcps_path_in + "fechas.csv")

In [ ]:
fechas_queries = pd.read_csv("fechas.csv")
fechas_queries = [datetime.datetime.strptime(x,"%Y_%m_%d" ) for x in fechas_queries.Fecha.values]

In [ ]:
def run_queries(bq,f,n_sample = 300000):
    # Funcion para correr todas las queries 
    bq.execute(f"DROP TABLE IF EXISTS {tabla_pivot2bq}")
    end_s = f.strftime("%Y_%m_%d")
    gcps_path_in_ = gcps_path_in + end_s + "/"
    today = f
    end_cal_test = f
    start_cal_test = end_cal_test - relativedelta(months=FORECAST_MONTHS)
    end_cal_train = start_cal_test - relativedelta(days=1)
    start_cal_train = end_cal_train - relativedelta(months=TRAINING_MONTHS)

    ETL_TABLES = []

    etl_full_table( gcps_path_in = gcps_path_in_, bq_app = bq, date_to=end_cal_train, date_from=start_cal_train, sample=True, n_sample = n_sample)
    size = bq.execute_response(f"SELECT COUNT(*) as size FROM {tabla_pivot2bq}")['size'][0]
    print(f"Size:{size}")
    
    # Target corre con start_cal_test y end_cal_test
    etl_target(gcps_path_in = gcps_path_in_, bq_app = bq,date_from = start_cal_test, date_to = end_cal_test, tables = ETL_TABLES)


    # Funciones que corren con queries para cada campo
    functions = [etl_IPT,etl_sellers,etl_orders_frequencies,etl_asp_pareto,etl_compras_shipping,etl_payments,etl_tarjetas,
                etl_locations,etl_demo,etl_installs,etl_visits]

    if(PAIS in ["MLA","MLB"]):        
        functions = functions + [etl_mp_app,etl_mp_mgm,etl_mp_prepaid]

    if(PAIS in ["MLB"]):
        functions = functions + [etl_reg_queries]

    for func in functions:
        func(gcps_path_in = gcps_path_in_, bq_app = bq,date_from = start_cal_train, date_to = end_cal_train, tables = ETL_TABLES)

    # Query autocontenida para realizar todos los join
    base_q = """SELECT PIVOT.CUS_CUST_ID,\n"""
    base_q = (base_q + "\n".join(f"""{t}.* EXCEPT (CUS_CUST_ID),""" for t in ETL_TABLES))[0:-1] #[0:-2] borra la ultima coma
    base_q = base_q + f"""\nFROM {tabla_pivot2bq} PIVOT\n"""
    base_q = (base_q + "\n".join(f"""LEFT JOIN {temp45_base_path + t} {t} \n\tON {t}.CUS_CUST_ID = PIVOT.CUS_CUST_ID""" for t in ETL_TABLES))
    create_q = f"""CREATE TABLE IF NOT EXISTS {temp45_base_path + 'SUMMARY_CAL_' + end_s} AS (\n{base_q}\n)"""
    bq.execute(create_q)
    print(ETL_TABLES)

In [ ]:
def join_tables(bq,fechas_queries,storage_folder):
    # Union de todas las tablas generadas por sampleo en una unica tabla de entrenamiento

    base_q = """SELECT * FROM\n"""
    union_q = """\nUNION ALL\n""".join([f"SELECT * FROM {temp45_base_path + 'SUMMARY_CAL_' + t.strftime('%Y_%m_%d')}" for t in fechas_queries])
    print(union_q)
    bq.fast_export(union_q,gcps_path_in + storage_folder)

    # Borrar tablas parciales generadas en la ultima parte del loop
    last_tables = list(bq.execute_response(f"""SELECT * FROM meli-marketing.TEMP45.INFORMATION_SCHEMA.TABLES
                    WHERE table_name LIKE '%{temp45_base_path.split('.')[-1]}%'""" )['table_name'])

    for table in last_tables:
        bq.execute(f"DROP TABLE IF EXISTS meli-marketing.TEMP45.{table}")

    # Borrar tablas summary generadas para este pais
    summary_tables = list(bq.execute_response(f"""SELECT * FROM meli-marketing.TEMP45.INFORMATION_SCHEMA.TABLES
                        WHERE table_name LIKE '%{PAIS}_ML_FORECAST_3M_SUMMARY_CAL_%'""" )['table_name'])

    for table in summary_tables:
        bq.execute(f"DROP TABLE IF EXISTS meli-marketing.TEMP45.{table}")

In [ ]:
start_time = time.monotonic()
#     for f in fechas_queries:
for f in fechas_queries:
    run_queries(bq,f)

    utils.prints3("Ready ",f)
join_tables(bq,fechas_queries,'summary_cal')

finish_time = (time.monotonic() - start_time)/60
print(f'ETL Time: {finish_time} minutes')
bqs.print_gcps("---------f'ETL Time: {finish_time} minutes'-----------",bq_log_path=bq_log_path,silent = True)

In [ ]:
# Drift detection dataset generation
# to_ = FECHA_END - relativedelta(months=FORECAST_MONTHS) - relativedelta(days=1)
# from_ = to_ - relativedelta(months = 2)

date_drift_check = FECHA_END
run_queries(bq,date_drift_check,n_sample = 1000000)
table = f"""{temp45_base_path + 'SUMMARY_CAL_' + date_drift_check.strftime("%Y_%m_%d")}"""
bq.fast_export(f"""SELECT * FROM {table}""",
               gcps_path_in + "summary_cal_drift")
bq.execute(f"DROP TABLE IF EXISTS {table}")


In [ ]:
def preprocess(gcps_path_in, folder_path):    
    
    summary_cal = read_gfs_parquet(gcps_path_in, bqs, folder_path)
    
    # Steps para preprocesar
    steps = []
    if (PAIS == 'MLB' or PAIS == 'MLA'):
        steps = steps + [prepaid,asset_mgm,install_mp]
    if (PAIS == 'MLB'):
        steps = steps + [reg_data]
    steps = steps + [orders_frequencies,IPT,locations,demograficos, compras_shipping, target, tarjetas, visitas_short,
         install_ml, payments, sellers, asp_pareto]

    # Traigo base de clientes como pivot para añadir todas las columnas

    
    for func in steps:
        start = time.time()
        bqs.print_gcps("--------------------",bq_log_path=bq_log_path,silent = True)
        bqs.print_gcps("Start time:",datetime.datetime.now(),bq_log_path=bq_log_path,silent = True)
        bqs.print_gcps("Starting:",func.__name__,"Summary size:",summary_cal.shape,bq_log_path=bq_log_path)
        summary_cal = func(bqs, gcps_path_in,summary_cal,"train")
        end = time.time()
        bqs.print_gcps("tiempo en resolver:", end-start,bq_log_path=bq_log_path,silent = True)
    
    summary_cal = generate_clusters(bqs,gcps_path_in,summary_cal,"train_train")
    summary_cal = pareto(bqs,gcps_path_in,summary_cal,"train_train") # Genero modelo forecast
    
    return summary_cal.copy()

In [ ]:
# Set fechas para descargar y joinear los datasets de entrenamiento

bqs.download_file(gcps_path_in + 'fechas.csv', 'fechas.csv')
fechas_queries = pd.read_csv('fechas.csv')
fechas_queries = [x for x in fechas_queries.Fecha.values]
summary_cal = preprocess(gcps_path_in, "summary_cal")


train_cols_by_segment = {0:[x for x in summary_cal.columns if x not in  ['CUS_CUST_ID', TARGET_COL,'GMV_TARGET']],
                    1:[x for x in summary_cal.columns if x not in  ['CUS_CUST_ID', TARGET_COL,'GMV_TARGET']],
                    2:[x for x in summary_cal.columns if x not in  ['CUS_CUST_ID', TARGET_COL,'GMV_TARGET']],
                    3:[x for x in summary_cal.columns if x not in  ['CUS_CUST_ID', TARGET_COL,'GMV_TARGET']]}

In [ ]:
# Calculo las variables drifteadas desde el dataset de entrenamiento (Today - 3 months) y el dataset
# del dia de hoy, para excluirlas del entrenamiento

summary_drift = preprocess(gcps_path_in, "summary_cal_drift")

In [ ]:
summary_cal_train_sample = {}
summary_cal_today_sample = {}
model_drifts = {}


for x in range(N_CLUSTERS):
    summary_cal_train_sample[x] = summary_cal[summary_cal.cluster == x].sample(15000, random_state = RANDOM_STATE)
    summary_cal_today_sample[x] = summary_drift[summary_drift.cluster == x].sample(15000, random_state = RANDOM_STATE)
    
    model_drifts[x] = Drift(setpoint_df = summary_cal_train_sample[x],
                            apply_df = summary_cal_today_sample[x],
                            target = TARGET_COL)
    model_drifts[x].apply_drift()
    EXCLUDE_VARIABLES_DRIFTED[x] = model_drifts[x].getDriftedTotal()

In [ ]:
def preparo_datos(summary_cal):
    
    numeric_dtypes = [np.dtype('float32'),np.dtype('float64'), np.dtype('int32'),np.dtype('int64')]

    total_cols = list(summary_cal.columns)
    if('CUS_CUST_ID' in total_cols): total_cols.remove('CUS_CUST_ID')
    
    # Evitar filtrar la variable target
    if(TARGET_COL in total_cols): total_cols.remove(TARGET_COL)
    
    # Eliminado de la query de Target, pero si lo vuelven a usar esta 
    if('GMV_TARGET' in total_cols): total_cols.remove('GMV_TARGET')
    
    
    train_cols_nocat = [x for x in total_cols if(summary_cal.dtypes[x] in numeric_dtypes)]
    
    cates = [x for x in total_cols if(summary_cal.dtypes[x] not in numeric_dtypes)]

    for x in train_cols_nocat:
        summary_cal[x] = summary_cal[x].astype(int)
    for x in cates:
        summary_cal[x] = summary_cal[x].astype(str)
    
    
    return summary_cal,cates,train_cols_nocat


In [ ]:
def get_latest_model(necessary_files,bqs):
    import re
    matches = {}
    bqs.create_clients()
    gs_bucket = bqs.gsClient.get_bucket('marketing-modelling') # me paro en mi bucket
    paths = gs_bucket.list_blobs(prefix="")
    for blob in paths:
        # Check si el pais esta y todos los archivos necesarios estan en esa ruta
        if ((f"{BU}/{INICIATIVA}/{PAIS}/model" in blob.name)): 
            match = re.search(r'\d{4}_\d{2}_\d{2}', blob.name).group()
            
            #Si contiene alguno de los archivos necesarios agrego la fecha al diccionario de cuenta de archivos
            if (any(file in blob.name for file in neccesary_files)):
                if match not in matches.keys():
                    matches[match] = 1
                else:
                    matches[match] = matches[match] + 1

    #print(matches)

    matches_all_files = [match for match in matches.keys() if (matches[match] == len(neccesary_files))]
    dates = [datetime.datetime.strptime(x,"%Y_%m_%d") for x in matches_all_files]
    if dates == []:
        latest_date = None
        latest_date_str = None
    else:
        latest_date = max(dates)
        latest_date_str = latest_date.strftime("%Y_%m_%d")


    return latest_date_str

## Hago seleccion de variables imporantes por segmento

### Entreno

In [ ]:
def modelo_by_segmento(summary_cal,l_train_cols, n_cluster, select_variables = False):
    summary_cal,cates,train_cols_nocat = preparo_datos(summary_cal[l_train_cols])
    #train, valid= train_test_split(summary_cal,test_size=0.15,random_state=0)
    train, valid= train_test_split(summary_cal[l_train_cols],test_size=0.15,random_state=0)
    train_full, test= train_test_split(train,test_size=0.2,random_state=0)
    
    if (select_variables):
        if(train_full.shape[0] > 1e6):
            train, _= train_test_split(train_full,test_size=0.6,random_state=0)  # reduzco para entrenamieto
        else:
            train = train_full
    else:
        if(train_full.shape[0] > 600000):
            train, _= train_test_split(train_full,train_size=500000,random_state=0)  # reduzco para entrenamieto
            print("resampleo de",train_full.shape,"a",train.shape[0])

        else:
            train = train_full
        
    print("Train:",train.shape[0],"Test:",test.shape[0],"Valid;",valid.shape[0])
    X_train_ =  np.c_[train[train_cols_nocat].values,train[cates].values]
    X_train_full =  np.c_[train_full[train_cols_nocat].values,train_full[cates].values]
    X_valid_ =  np.c_[valid[train_cols_nocat].values,valid[cates].values]
    X_test =  np.c_[test[train_cols_nocat].values,test[cates].values]

    y_train_full =  np.array(train_full[TARGET_COL])
    y_train_ =  np.array(train[TARGET_COL])
    y_valid_ =  np.array(valid[TARGET_COL])
    y_test =  np.array(test[TARGET_COL])  

    cates_ind = [X_train_.shape[1] - len(cates) + i for i in range(len(cates))]  # indices de las categorias

    # Entreno    
    basic_params = {
        
               'use_best_model':True,
               'loss_function': 'MAE',
                'eval_metric':'MAE',
                'verbose':False,
                'boosting_type':"Plain",
                'bootstrap_type':"Bernoulli",
                'iterations':500,
    #              'class_weights':weights_l,
    #              'rsm':0.1,  # OJO!!! esto solo va si tenes mas de 100 features!,
    #             'max_ctr_complexity':2,
               'random_state': 45
            }


    
    hyperparams = { 
                    'learning_rate':[0.01,0.05,0.1,0.2,0.3],
                    'depth': [1,3,5,7,10],                                                       
                    'l2_leaf_reg': [1,10,50,100,150,300,500,750,1000],
                    'border_count': [5,10,30,60,100,150,200]
                }

    d_types = { 'depth': int,
               'l2_leaf_reg': int,
               'border_count':int
              }

    if select_variables:
        
        params = train_combinations(X_train_,X_valid_,y_train_,y_valid_,"neg_mean_absolute_error",foo_predict_regression,
                              foo_model_regression,basic_params,d_types,hyperparams,cates_ind, n_iter = SELECT_VARIABLES_STEPS)
        best_model =  simple_fit(params,X_train_full,y_train_full,X_valid_,y_valid_,foo_model_regression,basic_params,d_types,cates_ind)
    
    else:
        params = train_combinations(X_train_,X_valid_,y_train_,y_valid_,"neg_mean_absolute_error",foo_predict_regression,
                          foo_model_regression,basic_params,d_types,hyperparams,cates_ind,log="training.txt", n_iter = FULL_TRAINING_STEPS)
        #params = train_combinations(X_train_,X_valid_,y_train_,y_valid_,"neg_mean_absolute_error",foo_predict_regression,
        #                  foo_model_regression,basic_params,d_types,hyperparams,cates_ind,log="training.txt")

        
        bqs.print_gcps("Ready Randomsearch",bq_log_path=bq_log_path)
        best_model =  simple_fit(params,X_train_full,y_train_full,X_valid_,y_valid_,foo_model_regression,basic_params,d_types,cates_ind)
        bqs.print_gcps("Ready best model",bq_log_path=bq_log_path)
        error = foo_evaluation_regression(y_test,foo_predict_regression(best_model,X_test))
        bqs.print_gcps("Error:",error,bq_log_path=bq_log_path)
    
    c = n_cluster
    imp = picture(np.r_[X_train_,X_valid_],np.r_[y_train_,y_valid_],X_test,y_test,cates_ind,best_model,foo_model_regression,train_cols_nocat,cates)
    bqs.upload_file("lgbmImportance.png", gcps_path_out + "lgbmImportance_"+str(c)+".png")
    
    if select_variables:
        imp.sort_values('Value', ascending = False).to_csv("importance_short_"+str(c)+".csv",index=False)
    else:
        imp.sort_values('Value', ascending = False).to_csv("importance_full_"+str(c)+".csv",index=False)
    
    if not select_variables:
        bqs.upload_file("importance_full_"+str(c)+".csv", gcps_path_out + "importance_"+str(c)+".csv")
    
    if select_variables:
        return best_model, 0
    else:
        return best_model, error

In [ ]:
# Defino columnas de entrenamiento para cada cluster, excluyendo las variables target (data leakage)
# y las variables que tuvieron un data drift detectado antes

train_cols_by_segment = {}

for c in range(N_CLUSTERS):
    train_cols_by_segment[c] = list(set(list(summary_cal.columns)) - \
        #set(['CUS_CUST_ID', TARGET_COL,'GMV_TARGET']) - \
        set(EXCLUDE_VARIABLES_DRIFTED[c])) 

In [ ]:
#Modelo reducido para la busqueda de variables importantes
for c in range(N_CLUSTERS):
        bqs.print_gcps("-------- ENTRENAMIENTO {} -------".format(c),bq_log_path=bq_log_path)

        #bqs.download_file(gcps_path_in + f'summary_train_{str(c)}.pkl', f'summary_train_{str(c)}.pkl')
        #summary_cal = pd.read_pickle(f'summary_train_{str(c)}.pkl')
        summary_cluster = summary_cal.loc[summary_cal.cluster == c]
        #Corro entrenamiento corto para el cluster seleccionado
        bqs.print_gcps(f"-------- VARIABLES EXCLUIDAS CLUSTER {c} : {model_drifts[c].getDriftedTotal()} -------", bq_log_path = bq_log_path)

        #model, error = modelo_by_segmento(summary_cluster,[x for x in summary_cluster.columns if x not in EXCLUDE_VARIABLES_DRIFT[x]], c, select_variables = True) #modelo para ese segmento
        model, error = modelo_by_segmento(summary_cluster,train_cols_by_segment[c], c, select_variables = True) #modelo para ese segmento
        with open("model_short_"+str(c)+".pkl", 'wb') as file: # guardo
            pickle.dump(model, file)

        gc.collect()

In [ ]:
# Busqueda de variables importancia en relacion a la variable 'random'
for n in range(N_CLUSTERS):
    imp = pd.read_csv(f'importance_short_{n}.csv').sort_values('Value', ascending = False)

    max_importance = imp.iloc[1,1]
    rand_importance = imp.loc[imp.Feature == 'rand'].values[0][1]

    print(f"{n}_max_importance: {max_importance}")
    print(f"{n}_rand_importance: {rand_importance}")

    mayores_a_rand = list(imp.loc[imp.Value > rand_importance].Feature)
    mayores_orden_4 = list(imp.loc[imp.Value >= (max_importance / 1000)].Feature)
    distinto_0 = list(imp.loc[imp.Value > 0.0].Feature)

    total_vars = [x for x in list(imp.Feature) if (x in mayores_a_rand) and (x in mayores_orden_4) and (x in distinto_0)]

    print(f" Variables Cluster {n}: {len(total_vars)}")
    pd.DataFrame(total_vars, columns = ['Feature']).to_csv(f"features_model_{n}.csv", index = False)

    # Subo features del modelo N al Storage, incluidos el cluster y la columna target
    bqs.upload_file(f"features_model_{n}.csv", gcps_path_out + f"features_model_{n}.csv")

In [ ]:
# Entrenamiento del modelo Full productivo
errores = []
for c in range(N_CLUSTERS):
    train_cols = train_cols_by_segment[c]
    bqs.print_gcps("-------- ENTRENAMIENTO {} -------".format(c), bq_log_path = bq_log_path)

    #bqs.download_file(gcps_path_in + "summary_train_"+str(c)+".pkl", "summary_train_"+str(c)+".pkl")
    #summary_cal = pd.read_pickle("summary_train_"+str(c)+".pkl")

    #Variables importantes procesadas en el paso previo + Target
    variables = list(pd.read_csv(f"features_model_{c}.csv").Feature) + [TARGET_COL]
    summary_cluster = summary_cal.loc[summary_cal.cluster == c]
    #summary_cal,cates,no_cates = preparo_datos(summary_cal)
    # Modelo para ese segmento (Cluster)
    model,error = modelo_by_segmento(summary_cluster,variables, c, select_variables = False)
    errores.append(error)
    bqs.print_gcps("-------- ERROR TRAINING {} -------".format(error), bq_log_path = bq_log_path)
    with open("model_"+str(c)+".pkl", 'wb') as file:
        pickle.dump(model, file)

    bqs.upload_file("model_"+str(c)+".pkl", gcps_path_out + "model_"+str(c)+".pkl")
    gc.collect()

In [ ]:
# from melitk.fda import workspace
# metrics_dict = {
#             'Error C0': errores[0],
#             'Error C1': errores[1],
#             'Error C2': errores[2],
#             'Error C3': errores[3]
#         }
    
# workspace.save_metrics(metrics_dict)

# import pickle
# df = pd.DataFrame([1,2,4,5],columns = ["hola"])

# serialized_dataset = pickle.dumps(df)
# workspace.save_raw_model(serialized_dataset)